In [8]:
import plotly.express as px
import pandas as pd
import time
from dash import Dash, html, dcc, dash_table, Input, Output
from arduino_iot_cloud import ArduinoCloudClient
from collections import deque
import plotly.graph_objs as go
import threading

# --- Device Credentials ---
DEVICE_ID = "8804f551-ab6a-4878-9b3d-9beae644d0e7"
SECRET_KEY = "nl49dbaA4#Y#hQe8N9JArkMyx"
USERNAME = DEVICE_ID

# --- Buffers for Live Data ---
MAX_WINDOW = 200
buffer_x = deque(maxlen=MAX_WINDOW)
buffer_y = deque(maxlen=MAX_WINDOW)
buffer_z = deque(maxlen=MAX_WINDOW)

# --- Dash App Layout ---
app = Dash()
app.layout = html.Div([
    html.H1("Smooth Live Accelerometer Graph"),
    
    html.P("Choose axis to display:"),
    dcc.Dropdown(
        id='axis-dropdown',
        options=[
            {'label': 'X Axis', 'value': 'x'},
            {'label': 'Y Axis', 'value': 'y'},
            {'label': 'Z Axis', 'value': 'z'},
            {'label': 'All (X, Y, Z)', 'value': 'all'}
        ],
        value='all'
    ),

    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=200, n_intervals=0),
    
    html.H3("Statistics"),
    dash_table.DataTable(
        id='stats-table',
        columns=[],
        data=[]
    ),

])

# --- Data Handlers ---
def handle_axis(axis, value):
    if axis == 'x':
        buffer_x.append(value)
    elif axis == 'y':
        buffer_y.append(value)
    elif axis == 'z':
        buffer_z.append(value)

def on_accelerometer_x_changed(client, value): handle_axis('x', value)
def on_accelerometer_y_changed(client, value): handle_axis('y', value)
def on_accelerometer_z_changed(client, value): handle_axis('z', value)

# --- Arduino Cloud in Thread ---
def arduino_thread():
    client = ArduinoCloudClient(
        device_id=DEVICE_ID,
        username=USERNAME,
        password=SECRET_KEY
    )
    client.register("py_x", value=None, on_write=on_accelerometer_x_changed)
    client.register("py_y", value=None, on_write=on_accelerometer_y_changed)
    client.register("py_z", value=None, on_write=on_accelerometer_z_changed)
    client.start()

# --- Live Graph Update ---
import numpy as np

@app.callback(
    Output('live-graph', 'figure'),
    Output('stats-table', 'data'),
    Output('stats-table', 'columns'),
    Input('interval-component', 'n_intervals'),
    Input('axis-dropdown', 'value')
)
def update_graph(n, selected_axis):
    # Make sure there is at least some data
    if not buffer_x or not buffer_y or not buffer_z:
        return go.Figure(), [], []

    # Get trimmed NumPy arrays (faster than list slicing)
    min_len = min(len(buffer_x), len(buffer_y), len(buffer_z))
    x = np.array(buffer_x)[-min_len:]
    y = np.array(buffer_y)[-min_len:]
    z = np.array(buffer_z)[-min_len:]

    fig = go.Figure()

    if selected_axis == 'all':
        fig.add_trace(go.Scatter(y=x, mode='lines', name='X'))
        fig.add_trace(go.Scatter(y=y, mode='lines', name='Y'))
        fig.add_trace(go.Scatter(y=z, mode='lines', name='Z'))
        fig.update_layout(title="Accelerometer XYZ Axes")
        stats_data, stats_columns = [], []
    else:
        data = {'x': x, 'y': y, 'z': z}[selected_axis]
        fig.add_trace(go.Scatter(y=data, mode='lines', name=selected_axis.upper()))
        fig.update_layout(title=f"{selected_axis.upper()} Axis")

        # Only compute stats when single axis is selected
        stats_series = pd.Series(data)
        stats_df = stats_series.describe().reset_index()
        stats_df.columns = ['Statistic', selected_axis.upper()]
        stats_data = stats_df.to_dict('records')
        stats_columns = [{"name": i, "id": i} for i in stats_df.columns]

    fig.update_layout(
        margin=dict(l=40, r=10, t=40, b=30),
        height=400,
        uirevision='static'  # keeps zoom and pan settings
    )

    return fig, stats_data, stats_columns


# --- Run ---
if __name__ == '__main__':
    threading.Thread(target=arduino_thread, daemon=True).start()
    app.run_server(debug=True)


ERROR:root:task: connection_task raised exception: .
